In [ ]:
sc

In [ ]:
from pyspark.mllib.linalg import Vector, Vectors
import numpy as np
from pyspark.mllib.stat import Statistics
import pandas as pd
from hdfs import Config
from pyspark.mllib.stat import Statistics
import time
from itertools import islice

SparkContext.setCheckpointDir(sc,"hdfs://ad-cluster-master:54310/user/checkpoints/")

######## CONFIG #######################
CLUSTER_ADD = "hdfs://ad-cluster-master:54310"
SOURCE_DIR = "/user/cluster/source_wfile/"
MODEL_DIR = "/user/cluster/"
num_cores = 6
gran = 3


#########################################################################
t1 = time.time()
client = Config().get_client()
FilesList = client.list(SOURCE_DIR)
#FilesList = FilesList[:3] #Debug
total = len(FilesList)

i=0
header = []


##### PRELOAD DOUBLE INDEX ######
rddFile = sc.textFile(CLUSTER_ADD + MODEL_DIR + "model.csv",  gran*num_cores)
rddT = rddFile.map(lambda x: tuple(x.split(","))).zipWithIndex().map(lambda x: (x[0][0], x[0][1],x[1]))

op = rddT.toDF(["date","action","uuid"])            

for file_ in FilesList:
    i=i+1 
    if i % 50 == 0:
        #op.persist(StorageLevel.MEMORY_AND_DISK)
        op.checkpoint()
        op.count()
        
    print("#########")
    print("Task: #" + str(i) + "/" + str(total))

    rddFile = sc.textFile(CLUSTER_ADD + SOURCE_DIR + file_,\
                          gran*num_cores)
    
    header.append(rddFile.take(1)[0])
    
    rddT = rddFile.zipWithIndex().filter(lambda col: col[1] > 0).map(lambda x: (map(float,x)[0],x[1]))

    df = rddT.toDF([header[i-1],"uuid"])
    op = op.join(df, "uuid")
    
t2 = time.time()
print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))

print("#############################")
print("FILE LOADING TASK FINISHED")
print("\n")

